# Melanoma Classification with MLP Model

In this notebook, a simple neural network based on the melanoma dataset will be build in order to assess initial performance.
The notebook works with data previously saved to the disk. To create this script, the model template was used.

The script that should be performed before this is the script: data_preparation_20192020.ipynb


<b>Running the script on cluster</b>
Please keep in mind the following
1. Load ALL images rather than a subset by changing to get_all_img_paths(img_folder) in Step 2 (get image paths 
2. The path to the data can be modified in Step 2 (get image paths + load data)
3. Make sure export is set to true
4. Save the results that are stored in the export folder and upload them seperately on google drive or github. The folder "export" is not tracked on github!!!

## 1. Set up

#### Set up for importing utilities

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

#### Install efficientnet

In [2]:
os.system("pip install efficientnet")

import efficientnet.tfkeras as efn

Defaulting to user installation because normal site-packages is not writeable


 #### Random State

In [3]:
from utilities import random_state
random_state = random_state() 

#### Export Config

In [4]:
export_results = True
on_cluster = False # there is some issue with h5py in the local env but it works on cluster so w/e
export_folder = f'Transfer_Learning/Cluster/091122-30percent-50'

date_format = "%d%m%Y%H%M%S" # timestamp format in exported files
if export_results:
    import datetime
    import os
    if not os.path.exists(export_folder): 
      os.makedirs(export_folder)
      print("Created new directory %s" %export_folder)

#### Resolution Setup

In [5]:
img_pixel = 224 # default : 128

#### Timer

Start the timer

In [6]:
import time
start = time.time()

## 2. Get data

#### Data config

In [7]:
downsampled_data = True
base_path = "data/30" 
current_train_melanoma_percentage = 0.3

#### Get image paths
For developing models on the cluster the max_images parameter should be removed. Instead call the method get_all_img_paths(img_folder).

In [8]:
from utilities import get_all_img_paths, get_img_paths

img_folder_train = base_path + "/train" + ("_downsampled" if downsampled_data else "")
img_folder_test = base_path + "/test" + ("_downsampled" if downsampled_data else "")

max_images_train = int(13653*0.5)
max_images_test = int(5804*0.5)

img_paths_train = get_img_paths(img_folder_train, max_images_train) 
img_paths_test = get_img_paths(img_folder_test, max_images_test)

#### Load data
Loads the images specified in img_paths into a data frame. This includes resizing the images and flattening them into an array and may take a while.

In [9]:
from utilities import load_train_test

groundtruth_file_train = base_path + "/ISIC_2020_2019_train" + ("_downsampled" if downsampled_data else "") + ".csv" 
groundtruth_file_test = base_path + "/ISIC_2020_2019_test" + ("_downsampled" if downsampled_data else "") + ".csv"


# available options
options = ["sequential", # first load train, then load test
           "parallel_train_test", # load train and test parallel (load data within train and test sequential)
           "sequential_train_test_parallel_chunks", # load first train, then test, but load the data within the sets parallel
           "parallel_fusion" # run train and test parallel and parallely load data with train and test 
          ]

# chose an option
option = "parallel_fusion"

df_train, df_test = load_train_test(img_paths_train, groundtruth_file_train, 
                                    img_paths_test, groundtruth_file_test, 
                                    option, img_pixel);

Num images per thread 463
start loading train
start thread #0 for train
start loading test
start thread #0 for test
start thread #1 for test
start thread #1 for train
start thread #2 for test
start thread #3 for test
start thread #2 for train
start thread #4 for test
start thread #3 for train
start thread #5 for test
start thread #4 for train
start thread #5 for train
start thread #6 for train
start thread #7 for train
start thread #8 for train
start thread #9 for train
start thread #10 for train
start thread #11 for train
start thread #12 for train
start thread #13 for train


KeyboardInterrupt: 

#### Split into target and predictors

In [ ]:
from utilities import split_predictors_target

X_train, y_train = split_predictors_target(df_train) 
X_test, y_test = split_predictors_target(df_test) 

In [ ]:
from utilities import unflatten_images_df

# I changed the shape of input data because of InceptionV3 requirements
X_train_flat = X_train
X_test_flat = X_test
X_train = unflatten_images_df(X_train, img_pixel=img_pixel)
X_test = unflatten_images_df(X_test, img_pixel=img_pixel)

#### Delete some no longer needed data

In [ ]:
del(df_train)
del(df_test)
del(img_paths_train)
del(img_paths_test)

## 3. Train model

Trying out a few transfer models Inception, VGG16, ResNet50, EfficientNetB0, EfficientNetB6.
The results are in \Transfer_Learning\transfer_learning_results.

In [ ]:
import sklearn 
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from tensorflow.keras.metrics import Recall
from tensorflow.keras.optimizers import RMSprop, Adam, SGD, Nadam
import keras

from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50



#from tensorflow.keras.applications.inception_v3 import InceptionV3
#base_model = InceptionV3(weights='imagenet', include_top=False, input_shape = (224,224,3)) 
#base_model.trainable = False
#base_model.layers.pop()
#base_model.layers.pop()
#base_model.layers.pop()
#add_model = Sequential()
#add_model.add(base_model)
#add_model.add(Flatten())
#add_model.add(Dense(1, activation='sigmoid'))

#base_model = VGG16(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
#base_model.trainable = False
#add_model = Sequential()
#add_model.add(base_model)
#add_model.add(Dense(512, activation='relu'))
#add_model.add(Dropout(0.5))
#add_model.add(Flatten())
#add_model.add(Dense(1, activation='sigmoid'))

#base_model. = ResNet50(include_top=False, weights='imagenet', pooling='max')
#base_model.trainable = False
#add_model = Sequential()
#add_model.add(base_model)
#add_model.add(Flatten())
#add_model.add(Dense(1, activation='sigmoid'))

#base_model = efn.EfficientNetB0(input_shape = (224,224, 3), include_top = False, weights = 'imagenet')
#base_model.trainable = False
#add_model = Sequential()
#add_model.add(base_model)
#add_model.add(Dense(1024, activation="relu"))
#add_model.add(Dropout(0.5))
#add_model.add(Flatten())
#add_model.add(Dense(1, activation='sigmoid'))
#model = add_model

#add_model = Sequential()
#base_model = efn.EfficientNetB0(input_shape = (224,224, 3), include_top = False, weights = 'imagenet')
#base_model.trainable = False
#add_model.add(base_model)
#add_model.add(Flatten())
#add_model.add(Dense(1, activation='sigmoid'))

#add_model = Sequential()
#base_model = efn.EfficientNetB6(input_shape=(224,224, 3),weights='imagenet',include_top=False)
#base_model.trainable = False
#add_model.add(base_model)
#add_model.add(GlobalAveragePooling2D())
#add_model.add(Dense(512, activation= 'relu'))
#add_model.add(Dropout(0.25))
#add_model.add(Flatten())
#add_model.add(Dense(1, activation='sigmoid'))


add_model = Sequential()
base_model = efn.EfficientNetB6(input_shape=(224,224, 3),weights='imagenet',include_top=False)
base_model.trainable = False
add_model.add(base_model)
add_model.add(Flatten())
add_model.add(Dense(1, activation='sigmoid'))


model = add_model

model.compile(loss='binary_crossentropy', optimizer=Nadam(), metrics=['accuracy', Recall(name="recall")])

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau 

melanoma_weight = (1/current_train_melanoma_percentage)/2
class_weight = {0: 1.,
                1: melanoma_weight,}

lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=5)

history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    batch_size=32, epochs=100,
                    callbacks=[EarlyStopping(patience=10), lr_scheduler])

In [ ]:
# print out model
model.summary()

In [ ]:
# store model
if export_results and on_cluster:
    model.save(export_folder + "/model_"+datetime.datetime.now().strftime(date_format)+".h5")

#### Timer
Stop the timer

In [ ]:
stop = time.time()
print(f'It took {stop - start} s to load the data and train the model')

if export_results:
    f = open(f'{export_folder}/overall_time.txt', 'w')
    f.write(f'Time it took : {stop - start} s')
    f.close()

## 4. Test model

#### Evaluate loss and accuracy during training

In [ ]:
# evaluate the model
_, train_acc, train_recall = model.evaluate(X_train, y_train)
_, test_acc, test_recall  = model.evaluate(X_test, y_test)

print('Accuracy\tTrain: %.3f, Test: %.3f' % (train_acc, test_acc))
print('Recall\tTrain: %.3f, Test: %.3f' % (train_recall, test_recall))

In [ ]:
from matplotlib import pyplot

_, axs = pyplot.subplots(3, 1, figsize=(20,15))

# plot loss during training
axs[0].plot(history.history['loss'], label='train')
axs[0].plot(history.history['val_loss'], label='test')
axs[0].set_title("Loss")
axs[0].legend()

# plot accuracy during training
axs[1].plot(history.history['accuracy'], label='train')
axs[1].plot(history.history['val_accuracy'], label='test')
axs[1].set_title("Accuracy")
axs[1].legend()

# plot recall during training
axs[2].plot(history.history['recall'], label='train')
axs[2].plot(history.history['val_recall'], label='test')
axs[2].set_title("Recall")
axs[2].legend()

if export_results:
    pyplot.savefig(export_folder + "/loss_and_accuracy_during_training_"+datetime.datetime.now().strftime(date_format)+".png")

pyplot.show()

#### Predict test set

In [ ]:
y_pred_continuous = model.predict(X_test)
y_pred_discrete = (model.predict(X_test) > 0.5).astype("int32")
y_pred = y_pred_discrete

#### Print report

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred, digits=4)
print(f'\nClassification_report=\n{report}')

if export_results:
    file = open(export_folder + "/classification_report_inception_"+datetime.datetime.now().strftime(date_format)+".txt", 'w')
    file.write(report)
    file.close()

#### Confusion Matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

class_names = ["no melanoma", "melanoma"]

cf = confusion_matrix(y_test, y_pred)
plot = sns.heatmap(cf, annot= True, fmt=".0f",
           xticklabels = class_names,
           yticklabels = class_names)
plot.set(xlabel='Prediction', ylabel='Actual')

if export_results:
    plot.get_figure().savefig(export_folder + '/confusion_matrix_' + datetime.datetime.now().strftime(date_format) + ".png")

#### Display images and predictions

In [ ]:
from utilities import display_results

X_train = X_train_flat
X_test = X_train_flat

plt_all = display_results(X_test, y_pred, y_test, 15, img_pixel)

if export_results:
    plt_all.savefig(export_folder + "/classification_results_"+datetime.datetime.now().strftime(date_format)+".png")

plt_all.show()

#### Display wrongly classified images

In [ ]:
from utilities import display_interesting_results

plt_wrong = display_interesting_results(X_test, y_pred, y_test, img_pixel=img_pixel)

if export_results:
    plt_wrong.savefig(export_folder + "/incorrect_classification_results_"+datetime.datetime.now().strftime(date_format)+".png")
    
plt_wrong.show()